In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# 1. 필요한 라이브러리 설치 및 임포트
# transformers 라이브러리가 설치되어 있지 않다면 아래 코드를 실행해주세요.
# !pip install transformers sentencepiece
import pandas as pd
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
import torch


In [ ]:

# 2. 파일 불러오기
# Colab에서 파일을 업로드한 후, 파일 경로를 지정합니다.
df = pd.read_csv('./data/NewsResult_re0806_no_square_rows.csv')



In [ ]:

# 3. KoBART 모델 및 토크나이저 불러오기
# "gogamza/kobart-summarization" 모델은 한국어 뉴스 요약에 특화되어 있습니다.
model_name = 'gogamza/kobart-summarization'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:

# GPU 사용 설정 (가능하다면)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Using device: {device}")

# 4. 요약 함수 정의
# KoBART 모델의 최대 토큰 길이는 1024이므로, 입력 길이에 유의해야 합니다.
def summarize_text(text):
    if not isinstance(text, str) or pd.isna(text):
        return ""

    # 텍스트가 모델의 최대 길이를 초과하는 경우, 앞부분을 자릅니다.
    # 더 긴 텍스트를 처리하려면 텍스트 분할 및 재요약 기법을 적용할 수 있습니다.
    input_ids = tokenizer.encode(text, max_length=1024, truncation=True, return_tensors='pt').to(device)

    # 요약문 생성
    summary_ids = model.generate(
        input_ids,
        length_penalty=1.0,  # 길이에 대한 패널티 (1.0이면 자연스러운 길이, 2.0이면 짧아짐)
        max_length=256,
        min_length=64,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return summary


Using device: cpu


In [ ]:

# 5. '처리된_본문_최종' 컬럼 요약 및 새 컬럼에 저장
df['요약문'] = df['처리된_본문_최종'].apply(summarize_text)



KeyboardInterrupt: 

In [ ]:
output_file_path = 'NewsResult_re0806_no_square_rows_summarized.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')